In [1]:
import numpy as np
import itertools
import math

actions = ['fold', 'call']
deck = ['J♥', 'J♠', 'Q♥', 'Q♠', 'K♥', 'K♠']
# deck = ['J♥', 'J♠', 'Q♥', 'Q♠', 'K♥', 'K♠', 'A♥', 'A♠']

cards = 4
chips_per_player = 5
rounds = 2

In [21]:
games = list(itertools.permutations(deck, cards))
info = [tuple(i) for i in np.array(np.meshgrid( range(chips_per_player*2), range(chips_per_player), range(rounds) )).T.reshape(-1,3)]
S = [(*g, *i) for g,i in list(itertools.product(games, info))]

def enum(s):
    return S.index(s)

def factor(i):
    return S[i]

print(len(S))
    
print(factor(3))
print(enum(('J♥', 'J♠', 'Q♥', 'Q♠', 1, 0, 0)))

36000
('J♥', 'J♠', 'Q♥', 'Q♠', 0, 3, 0)
5


In [58]:
def points(game):
    flush = {'J♥', 'K♥', 'Q♥'}
    seq = 'JKQ'
    pairs = ('KKQ', 'JKK', 'KQQ', 'JQQ', 'JJK', 'JJQ')
    # Flush
    if set(game) == flush:
        return 1
    
    cards = ''.join(sorted([card[0] for card in game]))
    # sequence
    if cards == seq:
        return 2
    
    return (3 + pairs.index(cards))


possible_games = {g:points(g) for g in list(itertools.permutations(deck, 3))}

my_game = ('Q♥', 'K♥', 'K♠')
possible_games[my_game]

3

In [61]:
def evaluate_game(s):
    p1,p2,t1,t2 = factor(s)[:4]
    p1_game = (p1,t1,t2)
    p2_game = (p2,t1,t2)

    points_p1 = possible_games[p1_game]
    points_p2 = possible_games[p2_game]

    return None if points_p1 == points_p2 else (points_p1 < points_p2)

In [68]:
s = enum(('Q♥', 'J♠', 'K♥', 'Q♠', 1, 0, 0))

evaluate_game(s)

False

In [28]:
def T(s,a,s_):
    p1,p2,t1,_,chips,pot,r = factor(s)
    p1_,p2_,t1_,_,chips_,pot_,r_ = factor(s_)
    
    lBound = lambda v: v if v>=0 else 0
    uBound = lambda v, b=6: v if v<=b else b

    if chips in (6,0) and r==0:
        return 0

    prob_round_1 =  1/(math.factorial(len(deck) ) / math.factorial(len(deck)-3))
    prob_round_2 =  1/(math.factorial(len(deck)-3) / math.factorial(len(deck)-3 - 1))

    if a == 0: # fold
        next_s = ((r_ == 0) and (pot_ == 0) and (chips_ == lBound(chips-1)))
        return prob_round_1 if next_s else 0
    
    if a == 1: # call
        if r == 0:
            next_s = (
                (r_ == r+1) and (pot_ == pot+1) and (chips_ == lBound(chips-1)) 
                and (p1==p1_), (p2==p2_), (t1==t1_)
            )
            return prob_round_2 if next_s else 0
        else:
            win = evaluate_game(s)
            prize = ((pot+1)*2 if win else (0, ((pot+1)*2)//2)[win is None])
            next_s = (
                (r_ == 0) and (pot_ == 0) and (chips_ == uBound(lBound(chips-1+prize)))
            )
            return prob_round_1 if next_s else 0
    

In [29]:
s = enum(('J♠', 'Q♠', 'Q♥', 'K♥', 4, 0, 0))
s_ = enum(('J♠', 'Q♠', 'Q♥', 'K♥', 3, 0, 0))

T(s,0,s_)

0.008333333333333333

In [30]:
s = enum(('J♠', 'Q♠', 'Q♥', 'K♥', 4, 0, 0))
s_ = enum(('J♠', 'Q♠', 'Q♥', 'K♥', 3, 1, 1))

T(s,1,s_)

0.3333333333333333

In [31]:
s = enum(('J♠', 'Q♠', 'Q♥', 'K♥', 4, 1, 1))
s_ = enum(('J♠', 'Q♠', 'Q♥', 'K♥', 6, 0, 0))

T(s,1,s_)

0.008333333333333333

In [33]:
s = enum(('J♠', 'J♥', 'Q♥', 'K♥', 4, 1, 1))
s_ = enum(('J♠', 'J♥', 'Q♥', 'K♥', 3, 0, 0))

T(s,1,s_)

0.008333333333333333

In [37]:
s = enum(('J♠', 'J♥', 'Q♥', 'K♥', 0, 0, 0))
s_ = enum(('J♠', 'J♥', 'Q♥', 'K♥', 0, 1, 1))

T(s,1,s_)

0